<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
response = requests.get(static_url, headers=headers)

# as an interactive example i want to know the methos and atributes of the response to se if it was succesful and what is the content
print(response)
print(response.status_code)
print(response.text[0:100])



<Response [200]>
200
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-la


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')
print(len(html_tables))


25


Starting from the third table is our target table contains the actual launch records.


In [15]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [24]:
header_cells

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date andtime ()
 </th>,
 <th scope="col"> 
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload
 </th>,
 <th scope="col">Payload mass
 </th>,
 <th scope="col">Orbit
 </th>,
 <th scope="col">Customer
 </th>,
 <th scope="col">Launchoutcome
 </th>,
 <th scope="col">
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">1
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">2
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">3
 </th>,
 <th rowspan="3" scope="row" style="text-align:center;">4
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">5
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">6
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">7
 </th>]

In [ ]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
header_cells = first_launch_table.find_all("th")
# Iterate each th element and apply the provided extract_column_from_header() to get a column name

for header_cell in header_cells:
    name = extract_column_from_header(header_cell)
    # Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
    if (name is not None and len(name) > 0):
        column_names.append(name)




['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [25]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [26]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [32]:
print(soup.find_all('table',"wikitable plainrowheaders collapsible")[6])

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-6"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch<br/>site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-6"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falc

In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            success_flag = True
            temp_data = {}

            try:
                temp_data['Flight No.'] = flight_number
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Flight No.': {e}")
                print(f"Row content: {row}")

            try:
                datatimelist = date_time(row[0])
                temp_data['Date'] = datatimelist[0].strip(',')
                temp_data['Time'] = datatimelist[1]
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Date' or 'Time': {e}")
                print(f"Row content: {row}")

            try:
                bv = booster_version(row[1])
                if not(bv):
                    bv = row[1].a.string
                temp_data['Version Booster'] = bv
                # print(bv) # Keep original print for debugging if needed
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Version Booster': {e}")
                print(f"Row content: {row}")

            try:
                temp_data['Launch site'] = row[2].a.string
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Launch site': {e}")
                print(f"Row content: {row}")

            try:
                temp_data['Payload'] = row[3].a.string
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Payload': {e}")
                print(f"Row content: {row}")

            try:
                temp_data['Payload mass'] = get_mass(row[4])
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Payload mass': {e}")
                print(f"Row content: {row}")

            try:
                temp_data['Orbit'] = row[5].a.string
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Orbit': {e}")
                print(f"Row content: {row}")

            try:
                temp_data['Customer'] = row[6].a.string
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Customer': {e}")
                print(f"Row content: {row}")

            try:
                temp_data['Launch outcome'] = list(row[7].strings)[0]
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Launch outcome': {e}")
                print(f"Row content: {row}")

            try:
                temp_data['Booster landing'] = landing_status(row[8])
            except Exception as e:
                success_flag = False
                print(f"Error extracting 'Booster landing': {e}")
                print(f"Row content: {row}")

            if success_flag:
                launch_dict['Flight No.'].append(temp_data['Flight No.'])
                launch_dict['Date'].append(temp_data['Date'])
                launch_dict['Time'].append(temp_data['Time'])
                launch_dict['Version Booster'].append(temp_data['Version Booster'])
                launch_dict['Launch site'].append(temp_data['Launch site'])
                launch_dict['Payload'].append(temp_data['Payload'])
                launch_dict['Payload mass'].append(temp_data['Payload mass'])
                launch_dict['Orbit'].append(temp_data['Orbit'])
                launch_dict['Customer'].append(temp_data['Customer'])
                launch_dict['Launch outcome'].append(temp_data['Launch outcome'])
                launch_dict['Booster landing'].append(temp_data['Booster landing'])
            else:
                print(f"Skipping row {extracted_row} due to extraction errors.")
            

Error extracting 'Customer': 'NoneType' object has no attribute 'string'
Row content: [<td>24 January 2021<br/>15:00<sup class="reference" id="cite_ref-624"><a href="#cite_note-624"><span class="cite-bracket">[</span>612<span class="cite-bracket">]</span></a></sup>
</td>, <td><a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">F9 B5</a> ♺<br/><a href="/wiki/List_of_Falcon_9_first-stage_boosters#B1058" title="List of Falcon 9 first-stage boosters">B1058.5</a><sup class="reference" id="cite_ref-625"><a href="#cite_note-625"><span class="cite-bracket">[</span>613<span class="cite-bracket">]</span></a></sup>
</td>, <td><a href="/wiki/Cape_Canaveral_Space_Force_Station" title="Cape Canaveral Space Force Station">CCSFS</a>,<br/><a href="/wiki/Cape_Canaveral_Space_Launch_Complex_40" title="Cape Canaveral Space Launch Complex 40">SLC-40</a>
</td>, <td><i><a class="new" href="/w/index.php?title=Transporter-1&amp;action=edit&amp;redlink=1" title="Transporter-1 (page does not exist)">Transp

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [40]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [43]:
for column in df.columns:
    print(f"Column: {column}")
    print(df[column].unique())
    print("-" * 30)

Column: Flight No.
['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '107' '108' '109' '110' '111' '112' '113'
 '114' '115' '116' '117' '118' '119' '120' '121']
------------------------------
Column: Launch site
['CCAFS' 'VAFB' 'Cape Canaveral' 'KSC' 'CCSFS']
------------------------------
Column: Payload
['Dragon Spacecraft Qualification Unit' 'Dragon' 'SpaceX CRS-1'
 'SpaceX CRS-2' 'CASSIOPE' 'SES-8' 'Thaicom 6' 'SpaceX CRS-3'
 'Orbcomm-OG2' 'AsiaSat 8' 'AsiaSat 6' 'SpaceX CRS-4' 'SpaceX CRS-5'
 'DSCOVR' 'ABS-3A'

In [46]:
for value in sorted(df['Version Booster'].unique()):
    print(value)

F9 B4B1039.2292
F9 B4B1040.2268
F9 B4B1041.2268
F9 B4B1043.2322
F9 B4B1045.2336
F9 B4[
F9 B5
F9 B5 ♺
F9 B5 ♺[
F9 B5311B1046.1268
F9 B5349B1048[
F9 B5B1046.2354
F9 B5B1046.3268
F9 B5B1047.2268
F9 B5B1047.3472
F9 B5B1048.2364
F9 B5B1048.3399
F9 B5B1049.2397
F9 B5B1049.3434
F9 B5B1049.6544
F9 B5B1049.9655
F9 B5B1051.10657
F9 B5B1051.2420
F9 B5B1051.6568
F9 B5B1051.8609
F9 B5B1056.2465
F9 B5B1056.3482
F9 B5B1058.2544
F9 B5B1058.3565
F9 B5B1058.8660
F9 B5B1060.2563
F9 B5B1060.6643
F9 B5B1060.7652
F9 B5B1061.2647
F9 B5B1063.2665
F9 B5B1067.1668
F9 B5[
F9 B5[]413
F9 FTB1029.2195
F9 FTB1031.2220
F9 FTB1032.2245
F9 FTB1035.2227
F9 FTB1036.2227
F9 FTB1038.2268
F9 FT[
F9 FT♺[
F9 v1.07B0003.18
F9 v1.07B0004.18
F9 v1.07B0005.18
F9 v1.07B0006.18
F9 v1.07B0007.18
F9 v1.1
F9 v1.17B10038
F9 v1.1[


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [41]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
